In [2]:
import os
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm

def parse_xml_annotations(xml_path):
    """解析单个XML标注文件，返回边界框列表"""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        boxes = []
        # 遍历所有目标
        for obj in root.iter('object'):
            bbox = obj.find('bndbox')
            if bbox is None:
                continue
            # 提取边界框坐标（xmin, ymin, xmax, ymax）
            xmin = float(bbox.find('xmin').text)
            ymin = float(bbox.find('ymin').text)
            xmax = float(bbox.find('xmax').text)
            ymax = float(bbox.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
        
        return boxes
    except Exception as e:
        print(f"解析XML出错 {xml_path}: {e}")
        return []

def generate_annotations_json(data_root, split):
    """生成指定split（Train/Test）的标注JSON文件"""
    # 定义路径
    annotations_dir = os.path.join(data_root, split, "Annotations")
    images_dir = os.path.join(data_root, split, "JPEGImages")
    output_path = os.path.join(data_root, f"inria_{split}_annotations.json")
    
    # 检查路径是否存在
    if not os.path.exists(annotations_dir):
        raise ValueError(f"标注文件夹不存在: {annotations_dir}")
    if not os.path.exists(images_dir):
        raise ValueError(f"图片文件夹不存在: {images_dir}")
    
    # 收集所有标注信息
    annotations_list = []
    xml_files = [f for f in os.listdir(annotations_dir) if f.endswith('.xml')]
    
    for xml_file in tqdm(xml_files, desc=f"处理{split}集"):
        # 获取文件名（不含扩展名）
        base_name = os.path.splitext(xml_file)[0]
        # 构建图片路径（支持png格式）
        image_path = os.path.join(images_dir, f"{base_name}.png")
        
        # 检查图片文件是否存在
        if not os.path.exists(image_path):
            print(f"警告: 图片文件不存在 {image_path}，跳过该标注")
            continue
        
        # 解析XML获取边界框
        xml_path = os.path.join(annotations_dir, xml_file)
        boxes = parse_xml_annotations(xml_path)
        
        # 添加到标注列表
        annotations_list.append({
            "image_path": image_path,  # 图片绝对路径或相对路径（根据实际使用场景调整）
            "boxes": boxes             # 边界框列表，每个框为[xmin, ymin, xmax, ymax]
        })
    
    # 保存为JSON文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(annotations_list, f, ensure_ascii=False, indent=2)
    
    print(f"成功生成 {output_path}，包含 {len(annotations_list)} 个样本")

if __name__ == "__main__":
    # 数据集根目录（根据实际路径修改）
    DATA_ROOT = "/opt/data/private/BlackBox/data/INRIAPerson"
    
    # 生成训练集和测试集标注
    generate_annotations_json(DATA_ROOT, "Train")
    generate_annotations_json(DATA_ROOT, "Test")

处理Train集: 100%|██████████| 614/614 [00:00<00:00, 743.89it/s]


成功生成 /opt/data/private/BlackBox/data/INRIAPerson/inria_Train_annotations.json，包含 614 个样本


处理Test集: 100%|██████████| 288/288 [00:00<00:00, 2001.36it/s]

成功生成 /opt/data/private/BlackBox/data/INRIAPerson/inria_Test_annotations.json，包含 288 个样本
